In [5]:
from numba import jit

In [14]:
class Node:
    def __init__(self, data, level, f_score, parent):
        self.data = data
        if parent == None:
            self.level = level
        else:
            self.level = level+1
        self.f_score = f_score
        self.parent = parent
    
    def find_index(self,num):
        index = self.data.find(num)
        i = index // 3
        j = index - ( i * 3 )
        return i,j
    
    def possible_moves(self):
        i, j = self.find_index("0")
        directions = [(-1,0),(1,0),(0,-1),(0,1)]
        directions_name = ["Up","Down","Left","Right"]
        all_directions = {directions[i]:directions_name[i] for i in range(4)}

        possible_moves = []
        for k in directions:
            if not ((i+k[0] < 0) or (i+k[0] > 2) or (j+k[1] < 0 ) or (j+k[1] > 2)):
                possible_moves.append(all_directions[k])
        return possible_moves
    
    def generate_children(self):
        directions = [(-1,0),(1,0),(0,-1),(0,1)]
        directions_name = ["Up","Down","Left","Right"]
        all_directions = {directions_name[i]:directions[i] for i in range(4)}
        
        possible_directions = self.possible_moves()
        
        i,j = self.find_index("0")
        
        move_to = []
        for d in possible_directions:
            move_to.append((i+all_directions[d][0], j+all_directions[d][1]))

        children = []
        for p in move_to:
            puzzle_list = list(self.data)

            zero_index = i * 3 + j
            dest_index = p[0] * 3 + p[1]

            puzzle_list[zero_index], puzzle_list[dest_index] = puzzle_list[dest_index], puzzle_list[zero_index]

            children.append("".join(puzzle_list))
        children_obj = []
        for i in children:
            children_obj.append(Node(i,self.level+1,0,self))
        return children_obj

In [13]:
class Puzzle:
    def __init__(self,puzzle):
        self.puzzle = puzzle
        self.open = []
        self.closed = []
        self.visited = {}
        
    def calculate_f(self, current, goal):
        return self.h_score(current.data,goal)+current.level
    
    def find_index(self, cur, num):
        index = cur.find(num)
        i = index // 3
        j = index - ( i * 3 )
        return i,j
    
    def h_score(self, current, goal):
        total = 0
        for i in range(len(current)):
            i,j = self.find_index(current, str(i))
            x,y = self.find_index(goal, str(i))
            total+= abs(i-x) + abs(j-y)
        for i in range(len(current)):
            if int(current[i])!=i+1:
                if int(current[i])==0 and i==8:
                    pass
                else:
                    total+=1
        return total
    
    def process(self):
        
        start = Node(self.puzzle,0,0,None)
        goal = "123456780"
        
        self.open.append(start)
        
        while True:
            cur = self.open[0]
            
            if(cur.data== goal):
                _list = []
                _list.append(cur)
                while cur.parent != None:
                    _list.append(cur.parent)
                    cur = cur.parent
                else:
                    _list.append(cur.parent)
                _list = _list[::-1]
                t = 0
                for i in _list:
                    if i != None:
                        for j in range(0,9,3):
                            print(i.data[j]," ",i.data[j+1]," ",i.data[j+2])
                        print("*********")
                    t+=1
                print("Total Steps:",t)
                break
            for i in cur.generate_children():
                i.fval = self.calculate_f(i,goal)
                if i.data not in self.visited or self.visited[i.data]>i.fval:
                    self.open.append(i)
                    self.visited[i.data]=i.fval
                
            self.closed.append(cur)
            self.visited[cur.data]=self.h_score(cur.data,goal)
            with open("log.txt", "w") as f:
                f.write(str(self.visited)+"\n")
            del self.open[0]
                
            self.open.sort(key = lambda x:x.fval,reverse=False)

In [12]:
puz = Puzzle("123046785")
puz.process()

UnsupportedError: Failed in object mode pipeline (step: analyzing bytecode)
The 'with (context manager) as (variable):' construct is not supported.

In [58]:
puz = Puzzle("102346758")
puz.process()

1   0   2
3   4   6
7   5   8
*********
1   2   0
3   4   6
7   5   8
*********
1   2   6
3   4   0
7   5   8
*********
1   2   6
3   0   4
7   5   8
*********
1   2   6
0   3   4
7   5   8
*********
0   2   6
1   3   4
7   5   8
*********
2   0   6
1   3   4
7   5   8
*********
2   3   6
1   0   4
7   5   8
*********
2   3   6
1   4   0
7   5   8
*********
2   3   0
1   4   6
7   5   8
*********
2   0   3
1   4   6
7   5   8
*********
0   2   3
1   4   6
7   5   8
*********
1   2   3
0   4   6
7   5   8
*********
1   2   3
4   0   6
7   5   8
*********
1   2   3
4   5   6
7   0   8
*********
1   2   3
4   5   6
7   8   0
*********
Total Steps: 17
